## Imports

In [ ]:
# Mathematics
import math
import random

# Scientific computing
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Progress bar
import tqdm

# Machine learning
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.tree import plot_tree
from sklearn.inspection import DecisionBoundaryDisplay

# Diabetes dataset
from sklearn.datasets import load_diabetes

# Data analysis and manipulation
import pandas as pd
pd.set_option('display.precision', 2) # 2 decimal places
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 100) # wide windows

## Ejemplo visto en clase: cultivo de papas

### Datos

In [ ]:
# Cargamos los datos
papas = pd.read_csv('Papas.csv')
papas

In [ ]:
papas.describe()

In [ ]:
papas.corr()

In [ ]:
scatter_plot = sns.scatterplot(data=papas, x="Lluvia", y="Rendimiento")
scatter_fig = scatter_plot.get_figure()
scatter_fig.savefig('Papas.png')

In [ ]:
# Pasamos a numpy para trabajar con sklearn
X = np.array(papas['Lluvia']).reshape(-1, 1)
y = np.array(papas['Rendimiento']).reshape(-1, 1)

### Regresión lineal

#### Regresión lineal simple

##### Entrenamiento

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X, y)

In [ ]:
y_pred = lin_reg.predict(X)

In [ ]:
plt.scatter(x=y,y=y_pred)
plt.xlabel('Verdad')
plt.ylabel('Predicción')
plt.title('Comparando verdad y predicción')
plt.show()

In [ ]:
# Coeficientes
w0 = lin_reg.intercept_
w1 = lin_reg.coef_
print(w0,w1)

In [ ]:
# Root mean squared error
np.sqrt(mean_squared_error(y,y_pred))

In [ ]:
# Coeficiente de determinación
lin_reg.score(X,y)

##### Gráfico de la hipótesis obtenida

In [ ]:
K=1
lin_reg_plot = sns.lmplot(x ="Lluvia", y ="Rendimiento", data = papas, order = K, ci = None, line_kws = {"color":"C1"})
plt.title("Regresión lineal con K="+str(K))
lin_reg_plot.savefig('Papas_LR_K_'+str(K)+'.png')

##### Curvas de nivel de la función de pérdida

In [ ]:
res = 300
lin_w0 = np.linspace(10.0, 20.0, res)
lin_w1 = np.linspace(0.0, 0.08, res)

In [ ]:
W0, W1 = np.meshgrid(lin_w0, lin_w1)

In [ ]:
L=np.zeros(W0.shape)

for i in range(res):
    for j in range(res):
        w0 = W0[i,j]
        w1 = W1[i,j]
        L[i,j] = np.sum(np.square(X*w1+w0-y))

In [ ]:
levels = [0, 50, 100, 150, 200.0, 250, 300.0, 350, 400, 500.0, 600.0, 700.0, 1200.0, 3000.0, 4000]
cp = plt.contour(W0, W1, L, levels, colors='black', linestyles='dashed', linewidths=1)
plt.clabel(cp, inline=1, fontsize=10)
cp = plt.contourf(W0, W1, L, levels)
plt.xlabel('w0')
plt.ylabel('w1')
plt.show()

#### Regresión lineal con polinomios

##### Entrenamiento

In [ ]:
# Elegimos el grado
K = 2

In [ ]:
# Obtenemos los features
poly = PolynomialFeatures(degree=K, include_bias=False)
X_poly = poly.fit_transform(X)

In [ ]:
X_poly

In [ ]:
# Escalamos
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_poly)

In [ ]:
X_norm

In [ ]:
poly_reg = LinearRegression()
poly_reg.fit(X_norm, y)

In [ ]:
y_pred = poly_reg.predict(X_norm)

In [ ]:
plt.scatter(x=y,y=y_pred)
plt.xlabel('Verdad')
plt.ylabel('Predicción')
plt.title('Comparando verdad y predicción')
plt.show()

In [ ]:
# Coeficientes
w0 = poly_reg.intercept_
w1 = poly_reg.coef_
print(w0,w1)

In [ ]:
# Root mean squared error
np.sqrt(mean_squared_error(y,y_pred))

In [ ]:
# Coeficiente de determinación
poly_reg.score(X_norm,y)

##### Gráfico de la hipótesis obtenida

In [ ]:
poly_reg_plot = sns.lmplot(x ="Lluvia", y ="Rendimiento", data = papas, order = K, ci = None, line_kws = {"color":"C1"})
plt.title("Regresión lineal con K="+str(K))
poly_reg_plot.savefig('Papas_LR_K_'+str(K)+'.png')

##### RMSE y tamaño de los coeficientes en función del grado

In [ ]:
sizes = []
rmses = []

for K in range(1,13):
    poly = PolynomialFeatures(degree=K, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    scaler = StandardScaler()
    X_norm = scaler.fit_transform(X_poly)
    
    poly_reg = LinearRegression()
    poly_reg.fit(X_norm, y)
    y_pred = poly_reg.predict(X_norm)
    rmses.append(np.sqrt(mean_squared_error(y,y_pred)))
    sizes.append(np.mean(np.abs(poly_reg.coef_)))

In [ ]:
rmses

In [ ]:
df_rmse = pd.DataFrame({"Grado":range(1,13), "Raíz MSE":rmses})
line_plot_rmse = sns.lineplot(
    data=df_rmse,
    x="Grado", y="Raíz MSE",
    marker='o',
    dashes=False,
    errorbar = ('ci', False)
)
plt.title("Raíz del MSE en función del grado")
line_plot_fig = line_plot_rmse.get_figure()
line_plot_fig.savefig('Papas_rmse.png')

In [ ]:
df_sizes = pd.DataFrame({"Grado":range(1,13), "Log valor abs promedio de coef":np.log(sizes)})
line_plot_sizes = sns.lineplot(
    data=df_sizes,
    x="Grado", y="Log valor abs promedio de coef",
    marker='o',
    dashes=False,
    errorbar = ('ci', False)
)
plt.title("Valor abs promedio de coef en función del grado")
line_plot_fig = line_plot_sizes.get_figure()
line_plot_fig.savefig('Papas_sizes.png')

### K vecinos más cercanos

#### Entrenamiento

In [ ]:
# Elegimos el k
k=1

In [ ]:
# Debemos elegir la distancia
knn_reg = KNeighborsRegressor(n_neighbors=k, metric='euclidean')

In [ ]:
knn_reg

In [ ]:
knn_reg.fit(X,y)

In [ ]:
y_pred = knn_reg.predict(X)

In [ ]:
plt.scatter(x=y,y=y_pred)
plt.xlabel('Verdad')
plt.ylabel('Predicción')
plt.title('Comparando verdad y predicción')
plt.show()

In [ ]:
# Root mean squared error
np.sqrt(mean_squared_error(y,y_pred))

#### Graficamos la hipótesis obtenida

In [ ]:
X_ax = np.array(range(50,378)).reshape(-1, 1)
pred_y = knn_reg.predict(X_ax)

In [ ]:
plt.step(X_ax, pred_y, lw=1.5, color="red", label="Predicción")
plt.scatter(X, y, s=5, color="blue", label="Verdad")
plt.legend()
plt.title('k vecinos más cercanos con k='+str(k))
plt.savefig('knn_Papas_k_'+str(k)+'.png')

#### RMSE en función de k

In [ ]:
rmses = []

for k in range(1,16):
    
    knn_reg = KNeighborsRegressor(n_neighbors=k, metric='euclidean')
    knn_reg.fit(X,y)
    
    y_pred = knn_reg.predict(X)
    rmses.append(np.sqrt(mean_squared_error(y,y_pred)))

In [ ]:
df_rmse = pd.DataFrame({"K":range(1,16), "Raíz MSE":rmses})
line_plot_rmse = sns.lineplot(
    data=df_rmse,
    x="K", y="Raíz MSE",
    marker='o',
    dashes=False,
    errorbar = ('ci', False)
)
plt.title("Raíz del MSE en función de K")
line_plot_fig = line_plot_rmse.get_figure()
line_plot_fig.savefig('knn_rmse.png')

#### Curse of dimensionality

In [ ]:
def random_point(dim):
    return np.array([random.random() for _ in range(dim)])

In [ ]:
def distance(v, w):
    """Calcula la distancia entre v y w"""
    return math.sqrt(np.sum(np.square(v-w)))

In [ ]:
def random_distances(dim, num_pairs):
    return [distance(random_point(dim), random_point(dim)) for _ in range(num_pairs)]

In [ ]:
dimensions = range(1, 101)
avg_distances = []
min_distances = []
random.seed(0)

for dim in tqdm.tqdm(dimensions, desc="Curse of Dimensionality"):
    distances = random_distances(dim, 10000) # 10,000 distancias random
    avg_distances.append(sum(distances) / 10000) # promedio
    min_distances.append(min(distances)) # minimo

In [ ]:
plt.plot(avg_distances, label = "Dist promedio")
plt.plot(min_distances, label = "Dist mínima")
plt.xlabel("Dimensión")
plt.title("10 mil distancias aleatorias")
plt.legend()
plt.savefig("knn_curse_1.png")

In [ ]:
ratio = [min_distances[i]/avg_distances[i] for i in range(len(avg_distances))]
plt.plot(ratio)
plt.xlabel("Dimensión")
plt.title("Ratio Dist mínima / Dist promedio")
plt.savefig("knn_curse_2.png")

### Árboles de decisión

#### Profundidad = 0

In [ ]:
plt.axhline(y=np.mean(y), lw=1.5, color="red", label="Predicción")
plt.scatter(X, y, s=5, color="blue", label="Verdad")
plt.legend()
plt.title('La mejor predicción constante es el promedio')
plt.savefig('constante_Papas.png')

#### Profundidad = 1

In [ ]:
X_ast = range(50,378)

rss = []

for x_ast in X_ast:
    c1 = papas.loc[papas['Lluvia']<x_ast,'Rendimiento'].mean()
    c2 = papas.loc[papas['Lluvia']>=x_ast,'Rendimiento'].mean()
    r = ((papas.loc[papas['Lluvia']<x_ast,'Rendimiento']-c1)**2).sum()+((papas.loc[papas['Lluvia']>=x_ast,'Rendimiento']-c2)**2).sum()
    rss.append(r)

In [ ]:
plt.step(X_ast,rss)
plt.axvline(x=((91+129)/2),linestyle='dashed',color='black')
plt.xlabel('Valor de x*')
plt.ylabel('RSS')
plt.title('¿Cuál es el mejor x*?')
plt.savefig('mejor_x_ast.png')

In [ ]:
def h(x):
    if x < (91+129)/2:
        return papas.loc[papas['Lluvia']<(91+129)/2,'Rendimiento'].mean()
    else:
        return papas.loc[papas['Lluvia']>=(91+129)/2,'Rendimiento'].mean()

In [ ]:
vec_h = [h(x) for x in X_ast]

In [ ]:
plt.step(X_ax, vec_h, lw=1.5, color="red", label="Predicción")
plt.scatter(X, y, s=5, color="blue", label="Verdad")
plt.legend()
plt.xlabel('Lluvia')
plt.ylabel('Rendimiento')
plt.title('La mejor predicción con una división')
plt.savefig('mejor_division.png')

#### Entrenamiento

In [ ]:
# Elegimos la profundidad
depth=1

In [ ]:
tree_reg = DecisionTreeRegressor(max_depth=depth)
tree_reg.fit(X, y)

In [ ]:
y_pred = tree_reg.predict(X)

In [ ]:
plt.scatter(x=y,y=y_pred)
plt.xlabel('Verdad')
plt.ylabel('Predicción')
plt.title('Comparando verdad y predicción')
plt.show()

In [ ]:
# Root mean squared error
np.sqrt(mean_squared_error(y,y_pred))

#### Graficamos la hipótesis obtenida

In [ ]:
X_ax = np.array(range(50,378)).reshape(-1, 1)
y_pred = tree_reg.predict(X_ax)

In [ ]:
# Plot the results
plt.figure()
plt.scatter(X, y, s=5, color="blue", label="Verdad")
plt.step(X_ax, y_pred, lw=1.5, color="red", label="Predicción")
plt.xlabel("Lluvia")
plt.ylabel("Rendimiento")
plt.title("Árbol de decisión con profundidad "+str(depth))
plt.legend()
plt.savefig('arbol_depth_'+str(depth)+'.png')

## Diabetes dataset

In [ ]:
diabetes = load_diabetes()

In [ ]:
# diabetes es un objeto tipo diccionario
diabetes.keys()

In [ ]:
# Tipos de los items
keys = list(diabetes.keys())
for k in range(len(keys)):
    print("Type of diabetes." + str(keys[k]) + " :", type(diabetes[keys[k]])) 

In [ ]:
# Item de descripción
print(diabetes.DESCR)

In [ ]:
# Nombres de las features o variables predictoras
print(diabetes.feature_names)

In [ ]:
# Extraemos los numpy arrays
X = diabetes.data 
y = diabetes.target

In [ ]:
print("Atributos de X")
print(
'''\
type: {}
dtype: {}
ndim: {}
shape: {}
size: {}
itemsize: {}
nbytes: {}\
'''.format(type(X),X.dtype,X.ndim,X.shape,X.size,X.itemsize,X.nbytes)
)

In [ ]:
# Primeras 10 filas
X[0:10,:]

In [ ]:
print("Atributos de y")
print(
'''\
type: {}
dtype: {}
ndim: {}
shape: {}
size: {}
itemsize: {}
nbytes: {}\
'''.format(type(y),y.dtype,y.ndim,y.shape,y.size,y.itemsize,y.nbytes)
)

In [ ]:
# Primeros 10 elementos
y[0:10]

### Manipulación usando Pandas

In [ ]:
# Seleccionamos bmi y bp como variables
X2 = X[:,2:4]

In [ ]:
# Convertimos a pandas dataframe 
df = pd.DataFrame(data=X2, columns=diabetes.feature_names[2:4])
df['target'] = pd.Series(y, dtype='float')

In [ ]:
# Tipo de objeto creado
type(df)

In [ ]:
# Número de filas en el data frame
len(df)

In [ ]:
# Número de filas y columnas en el data frame
df.shape

In [ ]:
# Visualización
df

In [ ]:
df.corr()

In [ ]:
sns.scatterplot(df, x='bmi',y='bp',hue='target')
plt.show()

### K vecinos más cercanos

In [ ]:
# Entrenamos
k=1
knn_reg = KNeighborsRegressor(n_neighbors=k, metric='euclidean')
knn_reg.fit(X2,y)

In [ ]:
y_pred = knn_reg.predict(X2)

In [ ]:
plt.scatter(x=y,y=y_pred)
plt.xlabel('Verdad')
plt.ylabel('Predicción')
plt.title('Comparando verdad y predicción')
plt.show()

In [ ]:
# Root mean squared error
np.sqrt(mean_squared_error(y,y_pred))

In [ ]:
# Graficamos la hipotesis obtenida
ax = plt.subplot(1, 1, 1)
disp=DecisionBoundaryDisplay.from_estimator(
    knn_reg,
    X2,
    response_method="predict",
    ax=ax,
    xlabel=diabetes.feature_names[2],
    ylabel=diabetes.feature_names[3],
    eps=0.02,
    grid_resolution = 1000,
    alpha=0.5, 
    cmap='Oranges'
    )

# Plotting the data points    
disp.ax_.scatter(X2[:, 0], X2[:, 1], 
                 c=y, edgecolor="k",
                 cmap='Oranges')
plt.show()

### Árboles de decisión

In [ ]:
# Entrenamos
depth=3
regtree = DecisionTreeRegressor(max_depth=depth)
regtree.fit(X2, y)

In [ ]:
y_pred = regtree.predict(X2)

In [ ]:
plt.scatter(x=y,y=y_pred)
plt.xlabel('Verdad')
plt.ylabel('Predicción')
plt.title('Comparando verdad y predicción')
plt.show()

In [ ]:
# Root mean squared error
np.sqrt(mean_squared_error(y,y_pred))

In [ ]:
# Graficamos el arbol
plt.figure(figsize=(24,6))
plot_tree(regtree,filled=True,fontsize=10)
plt.savefig('arbol_dim_2.png')

In [ ]:
# Graficamos la hipotesis obtenida
ax = plt.subplot(1, 1, 1)
disp=DecisionBoundaryDisplay.from_estimator(
    regtree,
    X2,
    response_method="predict",
    ax=ax,
    xlabel=diabetes.feature_names[2],
    ylabel=diabetes.feature_names[3],
    eps=0.02,
    grid_resolution = 1000,
    alpha=0.5, 
    cmap='Oranges'
    )

# Plotting the data points    
disp.ax_.scatter(X2[:, 0], X2[:, 1], 
                 c=y, edgecolor="k",
                 cmap='Oranges')
plt.savefig('boundary_dim_2.png')